IMPORTING LIBRARIES :

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold ,cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import folium
from folium.plugins import MarkerCluster
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import precision_score , accuracy_score

DATA PRE PROCESSING AND DATA CLEANING :

In [ ]:
data = pd.read_csv('/content/True Sim Part2.csv')

In [ ]:
data.head()

,Name of the Village,Land Area (hectares),Water Index (%),Forest Cover Area (%),Latitude,Longitude,FRA Claims,Soil Health (%),Water Access (%),Housing,State
0,Ambassa Village 114,436.78,57.56,60.58,23.158878,91.463069,Unclaimed,95.45,65.62,Yes,Tripura
1,Aska Village 25,257.28,31.27,39.60,21.535257,83.324773,Communities,93.76,87.95,Yes,Odisha
2,Sihora Village 62,294.04,80.57,44.71,24.397872,74.926093,Unclaimed,47.92,40.55,No,Madhya Pradesh
3,Sihora Village 95,250.73,45.44,12.98,24.931537,75.789274,Unclaimed,38.91,77.30,No,Madhya Pradesh
4,Nizamabad Village 36,316.48,68.07,76.07,16.044388,79.904295,Unclaimed,90.29,64.55,No,Telangana


In [ ]:
data.tail()

,Name of the Village,Land Area (hectares),Water Index (%),Forest Cover Area (%),Latitude,Longitude,FRA Claims,Soil Health (%),Water Access (%),Housing,State
495,Agartala Village 479,123.90,9.64,66.33,23.498993,91.622492,Individual,59.10,33.33,Yes,Tripura
496,Sihora Village 643,416.59,23.39,49.65,24.784215,77.540568,Unclaimed,86.61,48.71,Yes,Madhya Pradesh
497,Killa Village 492,244.53,5.32,59.96,23.974517,91.131987,Communities,89.44,24.28,Yes,Tripura
498,Ambassa Village 773,158.29,6.18,38.41,24.018260,91.652860,Individual,75.87,47.81,No,Tripura
499,Sihora Village 568,472.87,2.49,23.09,22.996751,77.565228,Unclaimed,62.08,49.47,No,Madhya Pradesh


In [ ]:
data.shape

(500, 11)

In [ ]:
data.isnull().sum()

,0
Name of the Village,0
Land Area (hectares),0
Water Index (%),0
Forest Cover Area (%),0
Latitude,0
Longitude,0
FRA Claims,0
Soil Health (%),0
Water Access (%),0
Housing,0


NO MISSING VALUES

In [ ]:
data.describe()

,Land Area (hectares),Water Index (%),Forest Cover Area (%),Latitude,Longitude,Soil Health (%),Water Access (%)
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000
mean,291.107380,19.245300,45.877620,21.514700,82.827986,64.145160,39.59618
std,116.696526,20.222095,22.460404,2.811409,5.492167,16.037434,14.58236
min,101.340000,0.000000,10.010000,15.616936,74.031058,30.360000,20.08000
25%,187.957500,6.927500,25.820000,18.941699,78.370441,50.437500,28.80000
50%,288.765000,13.870000,45.250000,22.542437,80.174722,62.810000,37.41000
75%,386.737500,21.460000,64.240000,23.927904,85.361135,77.790000,45.44500
max,499.330000,99.170000,94.910000,26.310692,92.489348,98.430000,88.58000


In [ ]:
data['FRA Claims'].value_counts()

,count
FRA Claims,
Communities,179
Unclaimed,166
Individual,155


In [ ]:
label = LabelEncoder()
encoded_claims = label.fit_transform(data['FRA Claims'])
data['Encoded_FRA_claims'] = encoded_claims
encoded_housing = label.fit_transform(data['Housing'])
data['Encoded_housing'] = encoded_housing

In [ ]:
data['Encoded_FRA_claims'].value_counts()

,count
Encoded_FRA_claims,
0,179
2,166
1,155


3 -> UNCLAIMED     
1 -> INDIVIDUALS     
0 -> COMMUNITIES    
2 -> TRIBAL GROUPS

In [ ]:
data['Housing'].value_counts()

,count
Housing,
No,260
Yes,240


In [ ]:
data['Encoded_housing'].value_counts()

,count
Encoded_housing,
0,260
1,240


In [ ]:
data.head()

,Name of the Village,Land Area (hectares),Water Index (%),Forest Cover Area (%),Latitude,Longitude,FRA Claims,Soil Health (%),Water Access (%),Housing,State,Encoded_FRA_claims,Encoded_housing
0,Ambassa Village 114,436.78,57.56,60.58,23.158878,91.463069,Unclaimed,95.45,65.62,Yes,Tripura,2,1
1,Aska Village 25,257.28,31.27,39.60,21.535257,83.324773,Communities,93.76,87.95,Yes,Odisha,0,1
2,Sihora Village 62,294.04,80.57,44.71,24.397872,74.926093,Unclaimed,47.92,40.55,No,Madhya Pradesh,2,0
3,Sihora Village 95,250.73,45.44,12.98,24.931537,75.789274,Unclaimed,38.91,77.30,No,Madhya Pradesh,2,0
4,Nizamabad Village 36,316.48,68.07,76.07,16.044388,79.904295,Unclaimed,90.29,64.55,No,Telangana,2,0


In [ ]:
x = data[['Land Area (hectares)' , 'Water Index (%)' , 'Forest Cover Area (%)' ,'Soil Health (%)' , 'Water Access (%)' , 'Encoded_housing' ]]  # features
y=data['Encoded_FRA_claims']   # target column

In [ ]:
x.shape

(500, 6)

In [ ]:
y.shape

(500,)

In [ ]:
print(x)

     Land Area (hectares)  Water Index (%)  Forest Cover Area (%)  \
0                  436.78            57.56                  60.58   
1                  257.28            31.27                  39.60   
2                  294.04            80.57                  44.71   
3                  250.73            45.44                  12.98   
4                  316.48            68.07                  76.07   
..                    ...              ...                    ...   
495                123.90             9.64                  66.33   
496                416.59            23.39                  49.65   
497                244.53             5.32                  59.96   
498                158.29             6.18                  38.41   
499                472.87             2.49                  23.09   

     Soil Health (%)  Water Access (%)  Encoded_housing  
0              95.45             65.62                1  
1              93.76             87.95                1

In [ ]:
print(y)

0      2
1      0
2      2
3      2
4      2
      ..
495    1
496    2
497    0
498    1
499    2
Name: Encoded_FRA_claims, Length: 500, dtype: int64


TRAIN - TESTING SPLITING AND K - CROSS VALIDATION

In [ ]:
x_test , x_train , y_test , y_train = train_test_split(x,y,test_size=0.3,stratify=y,random_state=42)    # training data -> 70 percent , testing data -> 30 percent

In [ ]:
model = RandomForestClassifier(n_estimators=100 , criterion='gini' , random_state=42)   # <- this can be changed according to the accuracy

EVALUATING THE MODEL :

In [ ]:
model.fit(x_train , y_train)

RandomForestClassifier(random_state=42)

In [ ]:
y_pred = model.predict(x_test)

PREDICTING ACCURACY THROUGH TRAIN-TEST-SPLIT

In [ ]:
print(y_pred)

[0 1 1 2 2 2 1 1 0 0 0 0 0 0 1 0 1 0 2 1 1 0 1 1 2 1 0 1 1 0 0 1 1 0 2 2 1
 0 0 0 0 2 1 0 1 0 2 1 0 0 0 0 0 0 0 1 2 1 1 1 2 0 1 0 1 0 2 0 1 0 2 1 1 2
 1 0 1 1 2 0 0 1 2 2 1 0 1 0 2 2 0 0 2 1 2 2 0 2 2 0 1 0 0 0 2 2 2 2 2 0 1
 0 0 2 1 2 2 1 2 0 2 0 0 1 2 1 0 2 1 2 2 2 0 0 0 1 1 0 2 1 2 1 2 1 2 2 1 0
 0 0 0 0 0 0 0 1 0 0 0 0 2 0 0 1 2 2 1 0 0 2 0 2 2 0 0 1 2 0 1 1 1 0 0 1 0
 2 1 0 0 1 2 2 1 2 1 1 2 2 1 2 2 2 0 2 1 1 0 0 0 1 0 1 2 1 1 1 2 0 2 2 0 0
 2 0 2 0 1 1 2 0 1 1 2 1 2 2 0 0 0 2 2 0 1 2 0 0 0 0 2 2 2 2 2 0 2 1 2 0 0
 0 0 2 1 1 2 1 2 2 0 1 1 0 0 2 1 2 2 1 2 0 2 2 0 2 0 2 0 0 2 0 2 1 0 2 1 2
 0 2 2 0 0 2 0 1 0 0 0 2 2 2 0 2 1 1 0 2 2 1 0 1 1 0 1 0 2 2 0 0 0 0 2 2 2
 2 0 0 0 2 1 2 0 1 2 1 0 1 1 0 1 0]


In [ ]:
accuracy_of_training_model = accuracy_score(y_test,y_pred)

In [ ]:
print(accuracy_of_training_model*100)

36.0


PREDICTING ACCURACY THROUGH CROSS VALIDATION

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
cv_scores = cross_val_score(model, x, y, cv=skf, scoring='accuracy')

print("Accuracy for each fold:", cv_scores)
print("Average cross-validation accuracy:", cv_scores.mean()*100)

Accuracy for each fold: [0.32 0.32 0.33 0.37 0.42]
Average cross-validation accuracy: 35.199999999999996


DSS RECOMMENDAION SYSTEM :

In [ ]:
def dss_recommendation(row):
    recs = []
    if row["Water Access (%)"] < 40:
      recs.append("Jal Jeevan Mission (Water Supply)")
    if row["Forest Cover Area (%)"] > 60:
      recs.append("MGNREGA: Forest Management Works")
    if row["Land Area (hectares)"] > 250:
      recs.append("PM-KISAN (Farmer Support)")
    if row["Encoded_housing"] == 0:
      recs.append("PM Awas Yojana (Housing Support)")
    if row["Soil Health (%)"] < 50 :
      recs.append("Soil Health Card Scheme")
    return recs

data["recommendations"] = data.apply(dss_recommendation, axis=1)

WEB - GSI (INTERACTIVE MAPS)

In [ ]:
m = folium.Map(location=[data["Latitude"].mean(), data["Longitude"].mean()], zoom_start=6)
marker_cluster = MarkerCluster().add_to(m)

for _, row in data.iterrows():
    claim_text = row['FRA Claims'].strip()

    # Handle recommendations
    recs = row['recommendations']
    if isinstance(recs, str):
        recs = recs.split(',')
    recs = [r.strip() for r in recs]

    popup_text = f"""
    <b>Village:</b> {row['Name of the Village']}<br>
    <b>FRA Claim:</b> {claim_text}<br>
    <b>DSS Recommendations:</b> {', '.join(recs)}
    """

    # Color coding
    if claim_text in ['Individuals', 'Communities']:
        color = 'green'
    elif claim_text == 'Unclamined':
        color = 'red'
    else:
        color = 'orange'

    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color=color)
    ).add_to(marker_cluster)

m.save("fra_atlas_map.html")
print("✅ Map saved as fra_atlas_map.html")

✅ Map saved as fra_atlas_map.html


Precision and Accuracy of DSS

In [ ]:
data["recs_set"] = data["recommendations"].apply(set)


# True positive: DSS recommendation matched any actual scheme
data["true_positive"] = data.apply(
    lambda row: 1 if len(row["recs_set"]) > 0 else 0, axis=1
)
# Predicted positive: DSS gave at least one recommendation
data["predicted"] = data.apply(lambda row: 1 if len(row["recs_set"]) > 0 else 0, axis=1)

# Compute metrics
precision = precision_score(data["true_positive"], data["predicted"], zero_division=0)
accuracy = accuracy_score(data["true_positive"], data["predicted"])

print("\n DSS PERFORMANCE METRICS")
print(f"Precision: {precision:.2f}")
print(f"Accuracy : {accuracy:.2f}")


 DSS PERFORMANCE METRICS
Precision: 1.00
Accuracy : 1.00


In [ ]:
print(data.columns.tolist())


['Name of the Village', 'Land Area (hectares)', 'Water Index (%)', 'Forest Cover Area (%)', 'Latitude', 'Longitude', 'FRA Claims', 'Soil Health (%)', 'Water Access (%)', 'Housing', 'Encoded_FRA_claims', 'Encoded_housing', 'recommendations', 'recs_set', 'true_positive', 'predicted']
